# Taller Práctico: Fundamentos de LangChain, Agentes e Inteligencia Artificial

## Bienvenida

Este taller te guiará paso a paso en el uso de LangChain, una de las herramientas más poderosas para construir aplicaciones con modelos de lenguaje grandes (LLMs). A través de ejemplos prácticos centrados en análisis meteorológico, aprenderás desde los conceptos básicos hasta la creación de agentes autónomos.

## Objetivos Educativos

- Comprender la arquitectura y filosofía de LangChain 1.0+
- Dominar los diferentes tipos de mensajes y prompts
- Aprender a crear y usar plantillas de prompts (PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate)
- Construir cadenas (chains) para procesar información de forma secuencial
- Implementar parsers para procesar y estructurar las salidas de los modelos
- Utilizar memoria conversacional en aplicaciones de IA
- Serializar y cargar prompts para reutilización
- Crear herramientas personalizadas para agentes
- Diseñar y ejecutar agentes autónomos
- Aplicar mejores prácticas en el desarrollo de aplicaciones con LLMs

## Resultados Esperados

Al finalizar este taller, serás capaz de:

- Construir aplicaciones conversacionales con memoria persistente
- Crear agentes que utilicen herramientas externas para resolver problemas complejos
- Diseñar prompts efectivos y reutilizables
- Procesar y validar las salidas de los modelos de lenguaje


## Prerrequisitos

- Conocimientos básicos de Python
- Cuenta de Google (para usar Google Colab)
- Disposición para experimentar y aprender

---

## 1. Instalación y Configuración Inicial

Antes de comenzar, necesitamos instalar las bibliotecas necesarias. LangChain es un framework modular, por lo que instalaremos los componentes específicos que utilizaremos.

### ¿Qué es LangChain?

LangChain es un framework diseñado para simplificar la creación de aplicaciones con modelos de lenguaje grandes. Proporciona abstracciones y herramientas que permiten:

- Conectar LLMs con fuentes de datos externas
- Mantener contexto conversacional
- Crear cadenas de procesamiento complejas
- Construir agentes que toman decisiones autónomas

En este taller usaremos Google Gemini, que ofrece una API gratuita para desarrollo en Google Colab.

In [ ]:
# Instalación de dependencias necesarias
# langchain-core: componentes básicos de LangChain
# langchain-google-genai: integración con modelos de Google Gemini
# langchain: framework principal

!pip install -q langchain langchain-core langchain-google-genai langchain-community

### Configuración de la API de Google Gemini

Google Colab proporciona acceso gratuito a través de `google.colab.userdata`. Para obtener tu API key:

1. Ve a [Google AI Studio](https://aistudio.google.com/app/apikey)
2. Crea una API key
3. En Colab, ve al ícono de llave (🔑) en el panel izquierdo
4. Agrega un secreto llamado `GOOGLE_API_KEY` con tu API key

In [ ]:
# Importamos las librerías necesarias
import os
from google.colab import userdata

# Configuramos la API key de forma segura
# En Colab: Click en la llave (🔑) en el panel izquierdo y agrega tu API key con el nombre "GOOGLE_API_KEY"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("Configuración completada correctamente")

---

## 2. Primer Contacto con LangChain y Gemini

Comencemos con la interacción más básica: enviar un mensaje a un modelo de lenguaje y recibir una respuesta.

### Concepto: Chat Model

Un Chat Model es una abstracción de LangChain que representa un modelo de lenguaje optimizado para conversaciones. A diferencia de los modelos de completado de texto, los chat models entienden el contexto de una conversación con mensajes de diferentes roles (usuario, asistente, sistema).

### Tipos de Mensajes

LangChain define varios tipos de mensajes:

- **SystemMessage**: Instrucciones iniciales que guían el comportamiento del modelo
- **HumanMessage**: Mensajes del usuario
- **AIMessage**: Respuestas del modelo
- **FunctionMessage** / **ToolMessage**: Mensajes relacionados con llamadas a funciones
- **ChatMessage**: Mensaje genérico con rol personalizado

In [ ]:
# Importamos el modelo de chat de Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Inicializamos el modelo
# Gemini 2.5 Flash es un modelo rápido y eficiente para la mayoría de tareas
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.7,  # Controla la creatividad (0 = determinista, 1 = muy creativo)
)

# Creamos un mensaje del usuario
mensaje = HumanMessage(content="¿Cuál es la temperatura ideal para el crecimiento de tomates?")

# Invocamos el modelo
respuesta = llm.invoke([mensaje])

print("Tipo de respuesta:", type(respuesta))
print("\nContenido:", respuesta.content)

### Ejemplo: Uso de SystemMessage

Los mensajes de sistema son fundamentales para definir el comportamiento y personalidad del modelo. Veamos cómo influyen en las respuestas.

In [ ]:
# Ejemplo con mensaje de sistema
mensajes = [
    SystemMessage(content="""Eres un meteorólogo experto especializado en análisis climático.
    Responde de forma técnica pero comprensible, siempre incluyendo datos científicos."""),
    HumanMessage(content="¿Por qué llueve más en algunas regiones que en otras?")
]

respuesta = llm.invoke(mensajes)
print("Respuesta del meteorólogo experto:")
print(respuesta.content)
print("\n" + "="*50 + "\n")

# Ahora sin mensaje de sistema
mensajes_simple = [
    HumanMessage(content="¿Por qué llueve más en algunas regiones que en otras?")
]

respuesta_simple = llm.invoke(mensajes_simple)
print("Respuesta sin contexto de sistema:")
print(respuesta_simple.content)

### Reflexión

Observa cómo el SystemMessage modifica el tono y profundidad de la respuesta. ¿Notaste diferencias en el estilo técnico entre ambas respuestas?

### Ejercicio Práctico 1

Modifica el SystemMessage del bloque anterior para que el modelo responda como un profesor de primaria explicando conceptos climáticos a niños de 8 años. Ejecuta nuevamente y compara las respuestas.

---

## 3. Plantillas de Prompts (Prompt Templates)

Las plantillas permiten crear prompts reutilizables con variables dinámicas. Esto es fundamental para mantener consistencia y facilitar el mantenimiento del código.

### 3.1 PromptTemplate (Plantillas Simples)

La clase PromptTemplate es la forma más básica de crear plantillas. Se usa principalmente para modelos de completado de texto, pero es importante entenderla como fundamento.

In [ ]:
from langchain_core.prompts import PromptTemplate

# Crear una plantilla simple con variables
# Las variables se definen con llaves: {nombre_variable}
template = """Eres un analista meteorológico.

Analiza las siguientes condiciones climáticas:
Ciudad: {ciudad}
Temperatura actual: {temperatura}°C
Humedad: {humedad}%
Presión atmosférica: {presion} hPa

Proporciona un análisis breve y profesional de estas condiciones."""

# Creamos la plantilla
prompt = PromptTemplate(
    input_variables=["ciudad", "temperatura", "humedad", "presion"],
    template=template
)

# Generamos un prompt específico con valores concretos
prompt_formateado = prompt.format(
    ciudad="Medellín",
    temperatura=22,
    humedad=75,
    presion=1013
)

print("Prompt generado:")
print(prompt_formateado)

### 3.2 ChatPromptTemplate (Plantillas para Conversaciones)

ChatPromptTemplate es la evolución para modelos de chat. Permite definir múltiples mensajes con diferentes roles y variables en cada uno.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Definir una plantilla de chat con múltiples mensajes
chat_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un {profesion} especializado en {especialidad}."),
    ("human", "¿Cuál es tu análisis sobre {tema}?"),
    ("ai", "Basándome en mi experiencia, puedo decirte que..."),
    ("human", "{pregunta_especifica}")
])

# Generar mensajes con valores específicos
mensajes = chat_template.format_messages(
    profesion="científico atmosférico",
    especialidad="fenómenos de El Niño y La Niña",
    tema="el impacto del cambio climático en los patrones de lluvia",
    pregunta_especifica="¿Qué podemos esperar en los próximos 10 años para Latinoamérica?"
)

# Veamos los mensajes generados
for i, mensaje in enumerate(mensajes):
    print(f"\nMensaje {i+1} ({mensaje.type}):")
    print(mensaje.content)
    print("-" * 50)

In [ ]:
# Ahora usemos esta plantilla con nuestro modelo
respuesta_final = llm.invoke(mensajes)
print("\nRespuesta del modelo:")
print(respuesta_final.content)

### 3.3 FewShotChatMessagePromptTemplate (Aprendizaje por Ejemplos)

El Few-Shot Learning es una técnica donde proporcionamos ejemplos al modelo para que entienda mejor el formato o estilo de respuesta que esperamos. Es especialmente útil para tareas estructuradas o con formato específico.

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# Definimos ejemplos de cómo queremos que el modelo responda
# Cada ejemplo es un diccionario con las variables y sus valores
ejemplos = [
    {
        "condicion": "cielo despejado, 28°C, baja humedad",
        "recomendacion": "APTO para actividades al aire libre. Hidratación importante. Protector solar recomendado. Nivel de riesgo: BAJO"
    },
    {
        "condicion": "lluvia intensa, 18°C, alta humedad",
        "recomendacion": "NO APTO para actividades al aire libre. Permanecer en interiores. Riesgo de inundaciones. Nivel de riesgo: ALTO"
    },
    {
        "condicion": "nublado, 22°C, humedad moderada",
        "recomendacion": "APTO con precauciones. Llevar paraguas por si acaso. Temperatura confortable. Nivel de riesgo: MEDIO"
    }
]

# Definir el formato de cada ejemplo
ejemplo_prompt = ChatPromptTemplate.from_messages([
    ("human", "Condiciones: {condicion}"),
    ("ai", "{recomendacion}")
])

# Crear el Few-Shot Prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=ejemplo_prompt,
    examples=ejemplos,
)

# Crear el prompt final que incluye los ejemplos y la nueva pregunta
prompt_final = ChatPromptTemplate.from_messages([
    ("system", "Eres un sistema de recomendaciones meteorológicas. Analiza las condiciones y proporciona recomendaciones claras siguiendo el formato de los ejemplos."),
    few_shot_prompt,
    ("human", "Condiciones: {condicion_nueva}")
])

# Probar con una nueva condición
respuesta = llm.invoke(
    prompt_final.format_messages(
        condicion_nueva="viento fuerte de 45 km/h, 15°C, cielo parcialmente nublado"
    )
)

print("Recomendación basada en ejemplos:")
print(respuesta.content)

### Ejercicio Práctico 2

Modifica el código anterior para crear un sistema de clasificación de fenómenos meteorológicos. Agrega tres ejemplos de fenómenos (tornado, tormenta eléctrica, nevada) con sus características y nivel de peligrosidad. Luego prueba clasificar: "vientos rotacionales de 120 km/h, nubes oscuras en forma de embudo".

---

## 4. Chains (Cadenas): Encadenamiento de Operaciones

Las cadenas son el corazón de LangChain. Permiten conectar múltiples componentes (prompts, modelos, parsers) en un flujo de procesamiento.

### Concepto: LCEL (LangChain Expression Language)

En LangChain 1.0+, usamos el operador `|` (pipe) para encadenar componentes. Esto crea flujos de datos claros y componibles.

In [ ]:
# Crear una cadena simple: Prompt -> Modelo
from langchain_core.output_parsers import StrOutputParser

# Definimos nuestro prompt
prompt_cadena = ChatPromptTemplate.from_messages([
    ("system", "Eres un analista de datos meteorológicos. Proporciona respuestas concisas y precisas."),
    ("human", "{consulta}")
])

# Creamos la cadena usando el operador pipe (|)
# El flujo es: entrada -> prompt -> modelo -> parser de string
cadena = prompt_cadena | llm | StrOutputParser()

# Ejecutar la cadena es tan simple como invocarla
resultado = cadena.invoke({"consulta": "¿Qué es la presión atmosférica y cómo se mide?"})

print(resultado)

### Cadenas Secuenciales

Podemos crear cadenas más complejas donde la salida de una se convierte en la entrada de la siguiente.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# Primera cadena: genera un pronóstico meteorológico
prompt_pronostico = ChatPromptTemplate.from_messages([
    ("system", "Eres un meteorólogo. Genera un pronóstico técnico breve para la ciudad solicitada."),
    ("human", "Ciudad: {ciudad}")
])

cadena_pronostico = prompt_pronostico | llm | StrOutputParser()

# Segunda cadena: traduce el pronóstico a lenguaje simple
prompt_simplificacion = ChatPromptTemplate.from_messages([
    ("system", "Eres un comunicador experto. Toma información técnica y explícala de forma que cualquier persona pueda entenderla."),
    ("human", "Información técnica: {pronostico_tecnico}")
])

cadena_simplificacion = prompt_simplificacion | llm | StrOutputParser()

# Combinar ambas cadenas
# RunnablePassthrough permite pasar datos entre cadenas
cadena_completa = (
    {"pronostico_tecnico": cadena_pronostico}
    | cadena_simplificacion
)

# Ejecutar la cadena completa
resultado_final = cadena_completa.invoke({"ciudad": "Bogotá"})

print("Pronóstico simplificado:")
print(resultado_final)

---

## 5. Output Parsers: Estructurando las Respuestas

Los parsers transforman la salida de texto del modelo en estructuras de datos útiles. Son fundamentales para integrar LLMs en aplicaciones reales.

### 5.1 StrOutputParser (Parser de String)

Ya lo hemos usado. Simplemente extrae el contenido de texto de la respuesta del modelo.

### 5.2 JsonOutputParser (Parser JSON)

Convierte la respuesta del modelo en un objeto JSON. Útil cuando necesitamos datos estructurados.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# Definimos la estructura que esperamos
class PronosticoMeteorologico(BaseModel):
    temperatura_max: int = Field(description="Temperatura máxima en grados Celsius")
    temperatura_min: int = Field(description="Temperatura mínima en grados Celsius")
    probabilidad_lluvia: int = Field(description="Probabilidad de lluvia en porcentaje")
    condicion: str = Field(description="Descripción de la condición climática")
    recomendacion: str = Field(description="Recomendación para el día")

# Crear el parser
parser = JsonOutputParser(pydantic_object=PronosticoMeteorologico)

# Crear prompt con instrucciones de formato
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un sistema meteorológico. Responde SIEMPRE en el formato JSON especificado.\n{format_instructions}"),
    ("human", "Genera un pronóstico para {ciudad} para mañana")
])

# Crear la cadena
cadena = prompt | llm | parser

# Ejecutar
resultado = cadena.invoke({
    "ciudad": "Cartagena",
    "format_instructions": parser.get_format_instructions()
})

print("Tipo de dato:", type(resultado))
print("\nDatos estructurados:")
for clave, valor in resultado.items():
    print(f"  {clave}: {valor}")

### 5.3 PydanticOutputParser (Parser con Validación)

Similar al JsonOutputParser pero con validación de tipos más estricta.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import List

class AlertaMeteorologica(BaseModel):
    tipo_alerta: str = Field(description="Tipo de alerta: amarilla, naranja o roja")
    fenomeno: str = Field(description="Fenómeno meteorológico")
    zonas_afectadas: List[str] = Field(description="Lista de zonas afectadas")
    duracion_horas: int = Field(description="Duración estimada en horas")
    medidas_precaucion: List[str] = Field(description="Medidas de precaución recomendadas")

# Crear parser
parser_pydantic = PydanticOutputParser(pydantic_object=AlertaMeteorologica)

# Crear prompt
prompt_alerta = ChatPromptTemplate.from_messages([
    ("system", "Eres un sistema de alertas meteorológicas. Genera alertas en el formato especificado.\n{format_instructions}"),
    ("human", "Genera una alerta para: {situacion}")
])

# Cadena
cadena_alerta = prompt_alerta | llm | parser_pydantic

# Ejecutar
alerta = cadena_alerta.invoke({
    "situacion": "Tormenta eléctrica severa aproximándose a la región andina",
    "format_instructions": parser_pydantic.get_format_instructions()
})

print("Objeto Pydantic generado:")
print(f"Tipo de alerta: {alerta.tipo_alerta}")
print(f"Fenómeno: {alerta.fenomeno}")
print(f"Zonas: {', '.join(alerta.zonas_afectadas)}")
print(f"Duración: {alerta.duracion_horas} horas")
print(f"Medidas de precaución:")
for medida in alerta.medidas_precaucion:
    print(f"  - {medida}")

### 5.4 CommaSeparatedListOutputParser

Útil cuando necesitamos listas simples de elementos.

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# Crear parser
parser_lista = CommaSeparatedListOutputParser()

# Prompt simple
prompt_lista = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto meteorológico."),
    ("human", "Lista 5 instrumentos usados para medir variables meteorológicas.\n{format_instructions}")
])

# Cadena
cadena_lista = prompt_lista | llm | parser_lista

# Ejecutar
instrumentos = cadena_lista.invoke({
    "format_instructions": parser_lista.get_format_instructions()
})

print("Tipo:", type(instrumentos))
print("\nInstrumentos meteorológicos:")
for i, instrumento in enumerate(instrumentos, 1):
    print(f"{i}. {instrumento}")

### Ejercicio Práctico 3

Crea una cadena que reciba el nombre de una ciudad y retorne un objeto JSON con: nombre_ciudad, latitud (aproximada), longitud (aproximada), zona_climatica, y temperatura_promedio_anual. Usa JsonOutputParser o PydanticOutputParser.

---

## 6. Memoria Conversacional

La memoria permite que las aplicaciones mantengan contexto entre múltiples interacciones. Es esencial para crear chatbots y asistentes conversacionales.

### 6.1 ConversationBufferMemory (Memoria de Buffer Simple)

Almacena todos los mensajes de la conversación en memoria.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage

# Crear memoria
memoria = ConversationBufferMemory(return_messages=True)

# Agregar mensajes a la memoria
memoria.chat_memory.add_user_message("¿Cuál es la temperatura promedio en el Amazonas?")
memoria.chat_memory.add_ai_message("La temperatura promedio en la región amazónica es de aproximadamente 26-28°C durante todo el año, con alta humedad.")

memoria.chat_memory.add_user_message("¿Y cuánto llueve allí?")
memoria.chat_memory.add_ai_message("La región amazónica recibe entre 2000 y 3000 mm de lluvia al año, siendo una de las zonas más lluviosas del planeta.")

# Ver el historial
print("Historial de conversación:")
for mensaje in memoria.chat_memory.messages:
    print(f"{mensaje.type}: {mensaje.content}\n")

### 6.2 ConversationBufferWindowMemory (Memoria con Ventana)

Mantiene solo los últimos N mensajes. Útil para limitar el consumo de tokens.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# Crear memoria con ventana de 2 interacciones (4 mensajes)
memoria_ventana = ConversationBufferWindowMemory(k=2, return_messages=True)

# Agregar varios mensajes
conversaciones = [
    ("¿Qué es un huracán?", "Un huracán es un sistema de baja presión con vientos superiores a 119 km/h."),
    ("¿Dónde se forman?", "Se forman en aguas tropicales cálidas, principalmente entre 5° y 20° de latitud."),
    ("¿Cuánto duran?", "Pueden durar desde días hasta semanas, dependiendo de las condiciones."),
    ("¿Son peligrosos?", "Sí, son extremadamente peligrosos por sus vientos, lluvias intensas e inundaciones.")
]

for pregunta, respuesta in conversaciones:
    memoria_ventana.chat_memory.add_user_message(pregunta)
    memoria_ventana.chat_memory.add_ai_message(respuesta)

# Verificar que solo mantiene los últimos 2 intercambios (4 mensajes)
print(f"Mensajes en memoria: {len(memoria_ventana.chat_memory.messages)}")
print("\nÚltimos mensajes recordados:")
for mensaje in memoria_ventana.chat_memory.messages:
    print(f"{mensaje.type}: {mensaje.content}")

### 6.3 ConversationSummaryMemory (Memoria con Resumen)

En lugar de almacenar mensajes completos, crea resúmenes. Muy útil para conversaciones largas.

In [ ]:
from langchain.memory import ConversationSummaryMemory

# Crear memoria con resumen (requiere un LLM para generar resúmenes)
memoria_resumen = ConversationSummaryMemory(llm=llm, return_messages=True)

# Simular una conversación larga sobre meteorología
conversacion_larga = [
    ("Explícame qué es la atmósfera",
     "La atmósfera es la capa de gases que rodea la Tierra, compuesta principalmente por nitrógeno (78%) y oxígeno (21%). Se divide en cinco capas: troposfera, estratosfera, mesosfera, termosfera y exosfera."),
    ("¿Qué sucede en la troposfera?",
     "La troposfera es la capa más baja, donde ocurren todos los fenómenos meteorológicos. Se extiende hasta aproximadamente 10-15 km de altura y contiene el 80% de la masa atmosférica."),
    ("¿Por qué es importante la estratosfera?",
     "La estratosfera contiene la capa de ozono, que absorbe la radiación ultravioleta dañina del sol, protegiendo la vida en la Tierra. Se extiende de 10 a 50 km de altura."),
]

for pregunta, respuesta in conversacion_larga:
    memoria_resumen.save_context(
        {"input": pregunta},
        {"output": respuesta}
    )

# Ver el resumen generado
print("Resumen de la conversación:")
print(memoria_resumen.load_memory_variables({}))

### 6.4 Uso de Memoria en Agentes

Para agentes, la memoria se integra de forma especial. Usaremos memoria de mensajes directamente en el historial del agente.

In [ ]:
# En agentes modernos, la memoria se maneja con un chat_history
# Veremos esto en detalle en la sección de agentes
# Por ahora, entendamos el concepto:

from langchain_core.chat_history import InMemoryChatMessageHistory

# Crear un historial de chat en memoria
historial_chat = InMemoryChatMessageHistory()

# Agregar mensajes
historial_chat.add_user_message("¿Qué temperatura hace hoy?")
historial_chat.add_ai_message("Hoy la temperatura es de 24°C con cielo parcialmente nublado.")
historial_chat.add_user_message("¿Lloverá?")
historial_chat.add_ai_message("Hay un 30% de probabilidad de lluvia en la tarde.")

# Ver mensajes
print("Historial completo:")
for mensaje in historial_chat.messages:
    print(f"  [{mensaje.type}]: {mensaje.content}")

---

## 7. Serialización de Prompts

La serialización permite guardar y cargar prompts, facilitando la reutilización y versionado.



### Guardar y cargar Prompts desde JSON

In [ ]:
# Si prefieres JSON, usa este enfoque:
import json
from langchain_core.prompts import ChatPromptTemplate

# 1. GUARDAR manualmente
prompt_meteorologia = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente meteorológico especializado.
    Tu trabajo es proporcionar información precisa y útil sobre el clima.
    Siempre incluye unidades de medida y contexto relevante."""),
    ("human", "{pregunta_usuario}")
])

# Convertir a formato serializable
prompt_dict = {
    "_type": "prompt",
    "input_variables": ["pregunta_usuario"],
    "messages": [
        {
            "type": "system",
            "content": """Eres un asistente meteorológico especializado.
    Tu trabajo es proporcionar información precisa y útil sobre el clima.
    Siempre incluye unidades de medida y contexto relevante."""
        },
        {
            "type": "human",
            "content": "{pregunta_usuario}"
        }
    ]
}

with open('prompt_meteorologia.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_dict, f, indent=2, ensure_ascii=False)

print("Prompt guardado en JSON")

# 2. CARGAR desde JSON
with open('prompt_meteorologia.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Reconstruir usando from_messages
mensajes = [(msg["type"], msg["content"]) for msg in data["messages"]]
prompt_reconstruido = ChatPromptTemplate.from_messages(mensajes)

# 3. Usar
respuesta = llm.invoke(
    prompt_reconstruido.format_messages(
        pregunta_usuario="¿Qué es la humedad relativa y cómo se relaciona con la sensación térmica?"
    )
)

print("\nRespuesta:")
print(respuesta.content)

### Reflexión

La serialización es crucial para:
- Mantener versiones de prompts en control de versiones
- Compartir prompts entre equipos
- Facilitar el testing A/B de diferentes versiones
- Separar la lógica del prompt del código de la aplicación

---

## 8. Herramientas (Tools)

Las herramientas permiten a los agentes interactuar con el mundo exterior: realizar cálculos, buscar información, ejecutar código, etc.

### 8.1 Concepto de Herramientas

Una herramienta es una función que el agente puede invocar. LangChain proporciona:
- Herramientas predefinidas (búsqueda web, calculadoras, APIs, etc.)
- Decoradores para crear herramientas personalizadas
- Integración con múltiples servicios

### 8.2 Crear Herramientas Personalizadas con @tool

In [ ]:
from langchain_core.tools import tool

# Herramienta 1: Convertir temperatura
@tool
def convertir_temperatura(temperatura: float, de_unidad: str, a_unidad: str) -> str:
    """Convierte temperatura entre Celsius, Fahrenheit y Kelvin.

    Args:
        temperatura: El valor numérico de la temperatura
        de_unidad: La unidad origen (celsius, fahrenheit, kelvin)
        a_unidad: La unidad destino (celsius, fahrenheit, kelvin)

    Returns:
        String con el resultado de la conversión
    """
    de_unidad = de_unidad.lower()
    a_unidad = a_unidad.lower()

    # Primero convertir todo a Celsius
    if de_unidad == "celsius":
        temp_celsius = temperatura
    elif de_unidad == "fahrenheit":
        temp_celsius = (temperatura - 32) * 5/9
    elif de_unidad == "kelvin":
        temp_celsius = temperatura - 273.15
    else:
        return "Unidad no reconocida"

    # Luego convertir de Celsius a la unidad destino
    if a_unidad == "celsius":
        resultado = temp_celsius
    elif a_unidad == "fahrenheit":
        resultado = (temp_celsius * 9/5) + 32
    elif a_unidad == "kelvin":
        resultado = temp_celsius + 273.15
    else:
        return "Unidad no reconocida"

    return f"{temperatura}°{de_unidad[0].upper()} = {resultado:.2f}°{a_unidad[0].upper()}"

# Herramienta 2: Calcular punto de rocío
@tool
def calcular_punto_rocio(temperatura: float, humedad_relativa: float) -> str:
    """Calcula el punto de rocío aproximado.

    Args:
        temperatura: Temperatura en grados Celsius
        humedad_relativa: Humedad relativa en porcentaje (0-100)

    Returns:
        String con el punto de rocío calculado
    """
    # Fórmula de Magnus simplificada
    a = 17.27
    b = 237.7

    alpha = ((a * temperatura) / (b + temperatura)) + (humedad_relativa / 100.0)
    punto_rocio = (b * alpha) / (a - alpha)

    return f"Con temperatura de {temperatura}°C y humedad de {humedad_relativa}%, el punto de rocío es aproximadamente {punto_rocio:.2f}°C"

# Herramienta 3: Clasificar índice UV
@tool
def clasificar_indice_uv(indice_uv: float) -> str:
    """Clasifica el índice UV y proporciona recomendaciones.

    Args:
        indice_uv: Valor del índice UV (0-11+)

    Returns:
        Clasificación y recomendaciones
    """
    if indice_uv <= 2:
        return f"Índice UV {indice_uv}: BAJO. No se requiere protección especial."
    elif indice_uv <= 5:
        return f"Índice UV {indice_uv}: MODERADO. Use protector solar. Busque sombra durante el mediodía."
    elif indice_uv <= 7:
        return f"Índice UV {indice_uv}: ALTO. Protección esencial. Use sombrero y gafas de sol."
    elif indice_uv <= 10:
        return f"Índice UV {indice_uv}: MUY ALTO. Protección extra. Evite exposición al sol entre 10am-4pm."
    else:
        return f"Índice UV {indice_uv}: EXTREMO. Evite la exposición solar. Permanezca en interiores si es posible."

# Probar las herramientas
print("Prueba de herramientas:\n")
print(convertir_temperatura.invoke({"temperatura": 25, "de_unidad": "celsius", "a_unidad": "fahrenheit"}))
print(calcular_punto_rocio.invoke({"temperatura": 25, "humedad_relativa": 60}))
print(clasificar_indice_uv.invoke({"indice_uv": 8.5}))

### 8.3 Anatomía de una Herramienta

Observa que cada herramienta tiene:

1. **Decorador @tool**: Convierte la función en una herramienta de LangChain
2. **Docstring detallado**: El modelo lo usa para entender cuándo y cómo usar la herramienta
3. **Type hints**: Especifica los tipos de los parámetros
4. **Lógica de negocio**: El código que ejecuta la herramienta
5. **Return claro**: Siempre retorna un string explicativo

### 8.4 Ver Información de las Herramientas

In [ ]:
# Ver los metadatos de una herramienta
print("Nombre de la herramienta:", convertir_temperatura.name)
print("\nDescripción:")
print(convertir_temperatura.description)
print("\nArgumentos esperados:")
print(convertir_temperatura.args)

### Ejercicio Práctico 4

Crea una herramienta llamada `calcular_sensacion_termica` que reciba temperatura y velocidad del viento, y calcule la sensación térmica usando la fórmula simplificada:

Sensación = Temperatura - (Velocidad_viento × 0.7)

Incluye un docstring completo y prueba tu herramienta.

---

## 9. Agentes: Razonamiento y Acción

Los agentes son sistemas que pueden razonar sobre qué hacer y tomar decisiones autónomas sobre qué herramientas usar.

### 9.1 ¿Qué es un Agente?

Un agente es un sistema que:
1. Recibe una tarea o pregunta
2. Razona sobre cómo resolverla
3. Decide qué herramientas usar
4. Ejecuta las herramientas necesarias
5. Analiza los resultados
6. Repite el proceso hasta resolver la tarea


### 9.2 Crear un Agente con LangChain

Vamos a crear un agente meteorológico que use nuestras herramientas.

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# Definir las herramientas que el agente puede usar
herramientas = [
    convertir_temperatura,
    calcular_punto_rocio,
    clasificar_indice_uv
]

# Crear el prompt para el agente
prompt_agente = ChatPromptTemplate.from_messages([
    ("system", """Eres un asistente meteorológico experto.
    Tienes acceso a herramientas especializadas para realizar cálculos y análisis.

    Cuando necesites realizar conversiones de temperatura, calcular punto de rocío,
    o evaluar índices UV, usa las herramientas disponibles.

    Siempre proporciona respuestas claras y profesionales."""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Crear el agente
agente = create_tool_calling_agent(
    llm=llm,
    tools=herramientas,
    prompt=prompt_agente
)

# Crear el executor del agente
agente_meteorologico = AgentExecutor(
    agent=agente,
    tools=herramientas,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,
)

print("Agente meteorológico creado exitosamente")

### 9.3 Ejecutar el Agente

Ahora pondremos al agente a trabajar con diferentes consultas.

In [ ]:
# Consulta 1: Uso de herramienta única
consulta1 = "Necesito saber cuántos grados Fahrenheit son 30 grados Celsius"

print("="*60)
print("CONSULTA 1:", consulta1)
print("="*60)

# CORRECCIÓN: AgentExecutor usa {"input": ...} no {"messages": ...}
resultado1 = agente_meteorologico.invoke({"input": consulta1})

# El resultado es un diccionario con la salida final
print("\n" + "="*60)
print("RESPUESTA FINAL:")
print("="*60)
print(resultado1["output"])

In [ ]:
# Consulta 2: Uso de múltiples herramientas
consulta2 = """Hoy la temperatura es de 28°C con 70% de humedad relativa,
y el índice UV es 9. Dame un reporte completo."""

print("\n\n" + "="*60)
print("CONSULTA 2:", consulta2)
print("="*60)

resultado2 = agente_meteorologico.invoke({"input": consulta2})

print("\n" + "="*60)
print("RESPUESTA FINAL:")
print("="*60)
print(resultado2["output"])

### 9.4 Agente con Memoria Conversacional

Agreguemos memoria para que el agente recuerde interacciones previas.

In [ ]:
from langchain.memory import ConversationBufferMemory

# Crear memoria conversacional
memoria = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Crear agente con memoria
agente_con_memoria = AgentExecutor(
    agent=agente,
    tools=herramientas,
    memory=memoria,
    verbose=True,
    handle_parsing_errors=True,
)

# Primera interacción
print("CONVERSACIÓN CON MEMORIA")
print("="*60)

pregunta1 = "La temperatura actual es 25°C. Conviértela a Fahrenheit."
print(f"\nUSUARIO: {pregunta1}")

resultado = agente_con_memoria.invoke({"input": pregunta1})
print(f"AGENTE: {resultado['output']}")

# Segunda interacción (el agente recuerda la temperatura anterior)
pregunta2 = "¿Cuál sería el punto de rocío con esa temperatura si la humedad es 65%?"
print(f"\nUSUARIO: {pregunta2}")

resultado2 = agente_con_memoria.invoke({"input": pregunta2})
print(f"AGENTE: {resultado2['output']}")

### Reflexión sobre Agentes

Observa cómo el agente:
1. **Entiende el contexto**: Interpreta lo que el usuario necesita
2. **Planifica**: Decide qué herramientas usar y en qué orden
3. **Ejecuta**: Llama a las herramientas con los parámetros correctos
4. **Sintetiza**: Combina los resultados en una respuesta coherente
5. **Recuerda**: Mantiene contexto entre interacciones (con memoria)



### Ejercicio Práctico 5

Crea una nueva herramienta llamada `evaluar_riesgo_lluvia` que reciba temperatura y humedad, y retorne un nivel de riesgo (bajo, medio, alto). Agrégala al agente y prueba con: "La temperatura es 22°C y la humedad 85%. ¿Hay riesgo de lluvia?"

---

## 10. Proyecto Final: Sistema Integrado de Análisis Meteorológico

Ahora que conoces todos los componentes, vamos a construir un sistema completo que integre todo lo aprendido.

In [ ]:
# Sistema completo de análisis meteorológico

# 1. Definir herramientas adicionales
@tool
def analizar_condiciones_completas(temperatura: float, humedad: float, presion: float, viento: float) -> str:
    """Analiza un conjunto completo de condiciones meteorológicas.

    Args:
        temperatura: Temperatura en grados Celsius
        humedad: Humedad relativa en porcentaje
        presion: Presión atmosférica en hPa
        viento: Velocidad del viento en km/h

    Returns:
        Análisis completo de las condiciones
    """
    analisis = []

    # Análisis de temperatura
    if temperatura < 10:
        analisis.append("Temperatura baja - abrigarse")
    elif temperatura > 30:
        analisis.append("Temperatura alta - mantenerse hidratado")
    else:
        analisis.append("Temperatura confortable")

    # Análisis de humedad
    if humedad > 80:
        analisis.append("Humedad muy alta - sensación de bochorno")
    elif humedad < 30:
        analisis.append("Humedad baja - posible sequedad")

    # Análisis de presión
    if presion < 1000:
        analisis.append("Presión baja - posible mal tiempo")
    elif presion > 1020:
        analisis.append("Presión alta - tiempo estable esperado")

    # Análisis de viento
    if viento > 40:
        analisis.append("Viento fuerte - precaución en exteriores")

    return "ANÁLISIS: " + " | ".join(analisis)

# 2. Crear conjunto completo de herramientas
herramientas_completas = [
    convertir_temperatura,
    calcular_punto_rocio,
    clasificar_indice_uv,
    analizar_condiciones_completas
]

# 3. Crear agente final con memoria
from langchain.memory import ConversationBufferMemory

memoria_sistema = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agente_sistema_completo = AgentExecutor(
    agent=agente,
    tools=herramientas_completas,
    memory=memoria_sistema,
    verbose=True,
    handle_parsing_errors=True,
)

print("Sistema de Análisis Meteorológico Completo - Inicializado")

In [ ]:
# 4. Función helper para interactuar con el sistema
def consultar_sistema(pregunta: str, sesion_id: str = "default"):
    """Función helper para consultar el sistema de forma más limpia"""
    resultado = agente_sistema_completo.invoke({"input": pregunta})
    return resultado["output"]

# 5. Probar el sistema completo
print("\n" + "="*70)
print("DEMO DEL SISTEMA COMPLETO")
print("="*70)

# Caso de uso 1
print("\n[Caso 1: Análisis integral]")
consulta = """Tengo las siguientes condiciones:
- Temperatura: 26°C
- Humedad: 75%
- Presión: 1015 hPa
- Viento: 25 km/h
- Índice UV: 7

Dame un análisis completo y recomendaciones."""

print(f"Usuario: {consulta}")
print(f"\nSistema: {consultar_sistema(consulta, 'sesion_demo_1')}")

# Caso de uso 2 (con memoria)
print("\n\n[Caso 2: Conversación con contexto]")
consulta2 = "¿Cuál sería el punto de rocío con esos valores de temperatura y humedad?"
print(f"Usuario: {consulta2}")
print(f"\nSistema: {consultar_sistema(consulta2, 'sesion_demo_1')}")

---

## 11. Conceptos Avanzados y Mejores Prácticas

### 11.1 Manejo de Errores en Herramientas

Es importante que las herramientas manejen errores gracefully.

In [ ]:
@tool
def herramienta_robusta(valor: float) -> str:
    """Ejemplo de herramienta con manejo de errores.

    Args:
        valor: Un valor numérico para procesar

    Returns:
        Resultado del procesamiento
    """
    try:
        if valor < 0:
            return "ERROR: El valor debe ser positivo"

        if valor > 100:
            return "ADVERTENCIA: Valor inusualmente alto, pero procesando..."

        resultado = valor * 2
        return f"Procesamiento exitoso: {resultado}"

    except Exception as e:
        return f"ERROR inesperado: {str(e)}"

# Probar
print(herramienta_robusta.invoke({"valor": 50}))
print(herramienta_robusta.invoke({"valor": -10}))
print(herramienta_robusta.invoke({"valor": 150}))

### 11.2 Optimización de Prompts

Consejos para crear prompts efectivos:

1. **Sea específico**: Define claramente el rol y comportamiento esperado
2. **Use ejemplos**: Few-shot learning mejora significativamente los resultados
3. **Estructure la salida**: Use parsers para obtener datos estructurados
4. **Itere y pruebe**: Los prompts mejoran con experimentación
5. **Considere el contexto**: Incluya información relevante pero concisa

### 11.3 Mejores Prácticas para Agentes

1. **Herramientas atómicas**: Cada herramienta debe hacer una cosa bien
2. **Descripciones claras**: Los docstrings son críticos para que el agente entienda cuándo usar cada herramienta
3. **Validación de entrada**: Siempre valide los parámetros de las herramientas
4. **Manejo de errores**: Retorne mensajes de error descriptivos
5. **Testing**: Pruebe cada herramienta independientemente antes de integrarla

---

## 12. Conclusiones y Próximos Pasos

### Lo que has aprendido

En este taller has dominado:

1. **Fundamentos de LangChain**: Arquitectura y componentes principales
2. **Mensajes y Prompts**: Diferentes tipos y cómo usarlos efectivamente
3. **Plantillas**: PromptTemplate, ChatPromptTemplate, y FewShotChatMessagePromptTemplate
4. **Cadenas**: Encadenamiento de operaciones con LCEL
5. **Output Parsers**: Estructuración de respuestas del modelo
6. **Memoria**: Manejo de contexto conversacional
7. **Serialización**: Guardar y cargar prompts
8. **Herramientas**: Creación y uso de herramientas personalizadas
9. **Agentes**: Toma de decisiones autónomas
10. **Integración**: Construcción de sistemas completos

### Aplicaciones Reales

Estos conceptos se aplican en:

- Chatbots empresariales con acceso a bases de datos
- Asistentes de análisis de datos
- Sistemas de automatización de tareas
- Herramientas de investigación y síntesis de información
- Interfaces conversacionales para APIs
- Sistemas de recomendación inteligentes

### Recursos para Continuar

- Documentación oficial de LangChain: https://python.langchain.com
- LangSmith para debugging y observabilidad: https://smith.langchain.com
- Comunidad de LangChain en Discord
- Repositorio de LangChain en GitHub

### Desafío Final

Construye un agente que:
1. Use al menos 4 herramientas personalizadas
2. Integre memoria conversacional
3. Maneje diferentes tipos de consultas
4. Procese salidas estructuradas con parsers
5. Incluya manejo de errores robusto

